# Parsing PDFs 

Pull in all of the PDF files and create objects for the text inside each one. 

This also probably should include a spell check

In [1]:
import PyPDF2 
from glob import glob

pdfs = glob('../pdfs/*.pdf') 

In [2]:
pdfs

['../pdfs/Cruz - 2010 - Chapter Six. Expanding The View The Challenges Of.pdf',
 '../pdfs/Cruz - 2010 - Index.pdf',
 '../pdfs/Frederiks and Nagy - 2016 - Religion, migration, and identity methodological .pdf',
 '../pdfs/Cruz - 2010 - Introduction.pdf',
 '../pdfs/Cruz - 2010 - Chapter One. Geographies Of Domestication Mapping.pdf',
 '../pdfs/cruz2010.pdf',
 '../pdfs/Cruz - 2010 - Chapter Two. Frontiers Of Struggle Negotiating Fi.pdf',
 '../pdfs/Cruz - 2010 - Conclusion.pdf',
 '../pdfs/Cruz - 2010 - Chapter Four. Exploring Theological Markers Delor.pdf',
 '../pdfs/Cruz - 2010 - An intercultural theology of migration pilgrims i.pdf',
 "../pdfs/Nguyen and Prior - 2014 - God's people on the move biblical and global pers.pdf",
 '../pdfs/Cruz - 2010 - Chapter Three. Expanding The Boundaries Theologic.pdf',
 '../pdfs/Gods People on the Move 2.pdf',
 '../pdfs/Cruz - 2010 - Chapter Five. A Different Cartography Mapping The.pdf',
 '../pdfs/Izuzquiza - 2011 - Breaking bread notes for a political t

# Preparing Texts 

## Lemmatizing and cleaning 

In [96]:
import nltk 
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as STOPWORDS 

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sgoodwin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/sgoodwin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [82]:
def text_clean(text):
    clean_list = []
    words = nltk.word_tokenize(text)
    for w in words:
        if w not in STOPWORDS:
            clean_list.append(lemmatizer.lemmatize(w))
    return clean_list

In [85]:
text_clean("The striped bats are hanging on their feet for best".lower())

['striped', 'bat', 'hanging', 'foot', 'best']

## Extracting Texts

In [17]:
pdf = open(pdfs[0], 'rb')
pdf_obj = PyPDF2.PdfFileReader(pdf)


In [18]:
print('No. of pages: {}'.format(pdf_obj.numPages))

No. of pages: 35


In [86]:
def pdf_extractor(pdf, corpus_list, text_list):
    '''Extract the text of pdfs and return a dictionary with
    the file name as a key, and the value being a list of the pages
    and the containing texts
    '''
    pdf_file_obj = open(pdf, 'rb')
    pdf_obj = PyPDF2.PdfFileReader(pdf_file_obj)
    for pn in range(0,pdf_obj.numPages):
        page = pdf_obj.getPage(pn)
        text = page.extractText().lower()
        cleaned_list = text_clean(text)
        corpus_list.append(cleaned_list)
        # corpus_list.append(page.extractText())
        text_list.append((pdf, pn))
        # if you want to create a dictionary
        # text_dict.setdefault(pdf, []).append(page.extractText())
    pdf_file_obj.close()
    return corpus_list, text_list

In [88]:
corpus_list = []
text_list = []

for pdf in pdfs:
    corpus_list, text_list = pdf_extractor(pdf, corpus_list, text_list)

# Creating LDA Model 



In [57]:
from gensim import corpora 
from gensim.models.ldamodel import LdaModel 

In [44]:
text = text_dict['../pdfs/Cruz - 2010 - Chapter Six. Expanding The View The Challenges Of.pdf']

In [103]:
def prepare_topic_model(corpus_list):
    corpus_dict = corpora.Dictionary(corpus_list)
    corpus_dict.filter_extremes(no_below=100, no_above=0.5)
    corpus = [corpus_dict.doc2bow(text) for text in corpus_list]
    lda_model = LdaModel(corpus=corpus, 
                        id2word=corpus_dict, num_topics=25,
                        random_state=100, update_every=1,
                        chunksize=100, passes=50,
                        alpha='symmetric', per_word_topics=True)
    return lda_model, corpus, corpus_dict

In [104]:
lda_model, corpus, corpus_dict = prepare_topic_model(corpus_list)

# Visualizing LDA Model

In [110]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [116]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, corpus_dict, mds='mmds')

of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.
 [_prepare.py:257]


In [118]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
16    -0.172723 -0.220506       1        1  10.950706
24    -0.405785 -0.134165       2        1  10.100261
4     -0.273454 -0.252547       3        1   8.418021
14     0.162144  0.011018       4        1   8.269429
23    -0.306124 -0.184880       5        1   6.535842
1      0.022208  0.111809       6        1   5.086045
12     0.007759 -0.449327       7        1   4.936429
7     -0.006962 -0.352134       8        1   4.816071
15     0.408644  0.136496       9        1   4.525380
13     0.218621  0.180757      10        1   4.374450
6      0.440091 -0.015744      11        1   4.326301
11     0.418646  0.359238      12        1   4.133867
2     -0.121199  0.216819      13        1   3.732332
17    -0.243858  0.285241      14        1   3.358915
0      0.260659 -0.294241      15        1   2.726360
20    -0.236133 -0.460476      16        1   2.369300
22    -0.424899  0.086261      17        1   2.327783
21    -0.488155  0.269838      18        1   1.969945
9      0.013309  0.427472      19        1   1.509415
18     0.323551 -0.438252      20        1   1.474855
3      0.430449 -0.167262      21        1   1.422291
10    -0.213052  0.508283      22        1   1.367979
19     0.193677  0.499724      23        1   1.101853
5     -0.003707 -0.061711      24        1   0.083088
8     -0.003707 -0.061711      25        1   0.083088, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
33    Default  2527.000000              ”  2527.000000  30.0000  30.0000
3     Default   862.000000              ;   862.000000  29.0000  29.0000
97    Default  1046.000000              %  1046.000000  28.0000  28.0000
32    Default  2168.000000              “  2168.000000  27.0000  27.0000
91    Default   932.000000         church   932.000000  26.0000  26.0000
82    Default   597.000000              !   597.000000  25.0000  25.0000
83    Default   874.000000              ?   874.000000  24.0000  24.0000
78    Default   952.000000            dhs   952.000000  23.0000  23.0000
17    Default   886.000000      migration   886.000000  22.0000  22.0000
114   Default   479.000000              &   479.000000  21.0000  21.0000
59    Default   932.000000        migrant   932.000000  20.0000  20.0000
53    Default  1846.000000              ’  1846.000000  19.0000  19.0000
26    Default   625.000000       theology   625.000000  18.0000  18.0000
37    Default   787.000000              e   787.000000  17.0000  17.0000
29    Default  1111.000000          woman  1111.000000  16.0000  16.0000
23    Default   555.000000       struggle   555.000000  15.0000  15.0000
11    Default   455.000000         eology   455.000000  14.0000  14.0000
13    Default   677.000000       filipino   677.000000  13.0000  13.0000
47    Default   975.000000              s   975.000000  12.0000  12.0000
39    Default   450.000000            god   450.000000  11.0000  11.0000
18    Default   569.000000            new   569.000000  10.0000  10.0000
30    Default   486.000000         worker   486.000000   9.0000   9.0000
106   Default   373.000000           hong   373.000000   8.0000   8.0000
107   Default   372.000000           kong   372.000000   7.0000   7.0000
5     Default   475.000000      christian   475.000000   6.0000   6.0000
69    Default   384.000000       religion   384.000000   5.0000   5.0000
54    Default   221.000000              5   221.000000   4.0000   4.0000
61    Default   473.000000         people   473.000000   3.0000   3.0000
21    Default   294.000000          press   294.000000   2.0000   2.0000
92    Default   345.000000      community   345.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
100   Topic25     0.307105        mission   178.614243   0.7272  -4.7791
85    Topic25    